In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer

df = pd.read_csv("data/train.csv")

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

In [ ]:
X = df.drop(["PassengerId", "Survived", "Name"], axis="columns")
y = df["Survived"]

In [ ]:
X_enc = pd.get_dummies(X, columns = ["Pclass", "Sex", "Embarked"])

has_cabin = [int(pd.isna(cabin)) for cabin in X_enc["Cabin"]]
X_enc["has cabin"] = has_cabin

X_enc = X_enc.drop(["Cabin", "Ticket"], axis = "columns")

In [ ]:
imputer = SimpleImputer(missing_values=pd.NA, strategy="most_frequent")
X_enc["Age"] = imputer.fit_transform(np.array(X_enc["Age"]).reshape(-1,1))

X_enc.head(10)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(kernel="rbf", C=100))
])

X_train, X_test, y_train, y_test = train_test_split(X_enc, y, random_state=12)

from sklearn.model_selection import GridSearchCV

parameters = {
    "C": [100, 200, 300],
    "kernel": ["linear", "rbf"],
    "gamma": ['scale', 1, 0.1, 0.01, 0.001]
}

optimal = GridSearchCV(estimator = SVC(), param_grid = parameters, cv=10)

optimal.fit(X_train_scaled, y_train)

print(optimal.best_params_)


In [ ]:
pipe.fit(X_train, y_train)
predictions = pipe.predict(X_test)

print(accuracy_score(y_test, predictions))